In [44]:
import pandas as pd
import numpy as np

stox_1 = pd.read_csv('Equity Valuation CSVs/MSCIEmergingMarketsPBRatio.csv')
stox_2 = pd.read_csv('Equity Valuation CSVs/MSCIEuropePBRatio.csv')
stox_3 = pd.read_csv('Equity Valuation CSVs/SP500PBRatio.csv')

In [45]:
def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

In [46]:
dates = intersection(stox_1['Date'], stox_3['Date'])

In [47]:
stox_1 = stox_1[stox_1['Date'].isin(dates)]
stox_2 = stox_2[stox_2['Date'].isin(dates)]
stox_3 = stox_3[stox_3['Date'].isin(dates)]

In [48]:
stox_1['diff'] = stox_1[' MSCI EM P/B Ratio'].diff()
stox_2['diff'] = stox_2['MSCI EU Index P/B ratio'].diff()
stox_3['diff'] = stox_3['S&P 500 Index P/B Ratio'].diff()

In [49]:
def roll(date, data, col, n):
    # data is a df
    # col is a column name in df
    
    index = data['Date'].loc[data['Date']==date].index.values[0]
    last_index = data.index[len(data)-1]
    
    if index + n < len(data):
        previous = data.loc[index:index+n]
        avg = np.sum(previous[col].values)/len(previous)
    else:
        previous = data.loc[index:last_index]
        avg = np.sum(previous[col].values)/len(previous)
        
    return avg

In [50]:
stox_1_roll = []
stox_2_roll = []
stox_3_roll = []

for index in stox_1.index:
    date = stox_1.at[index, 'Date']
    stox_1_roll.append(roll(date, stox_1, 'diff', 10))
    
for index in stox_2.index:
    date = stox_2.at[index, 'Date']
    stox_2_roll.append(roll(date, stox_2, 'diff', 10))
    
for index in stox_3.index:
    date = stox_3.at[index, 'Date']
    stox_3_roll.append(roll(date, stox_3, 'diff', 10))

In [51]:
stox = pd.DataFrame({'Date':stox_1['Date'],
                     'stox_1_roll':stox_1_roll,
                     'stox_2_roll':stox_2_roll,
                     'stox_3_roll':stox_3_roll})

In [52]:
def into_bucket(a, value):
    b=np.round(a*value)/value+1-1
    return b

In [53]:
import statistics
avg_1 = statistics.mean(stox['stox_1_roll'].dropna())
avg_2 = statistics.mean(stox['stox_2_roll'].dropna())
avg_3 = statistics.mean(stox['stox_3_roll'].dropna())

sd_1 = statistics.stdev(stox['stox_1_roll'].dropna())
sd_2 = statistics.stdev(stox['stox_2_roll'].dropna())
sd_3 = statistics.stdev(stox['stox_3_roll'].dropna())

zscores_1 = np.zeros([4,len(stox.index)])
zscores_2 = np.zeros([4,len(stox.index)])
zscores_3 = np.zeros([4,len(stox.index)])

for a in range(1,5):
    for i in range(len(stox)):
        if stox['stox_1_roll'][stox.index[i]] != 'NaN':
            z_bucket = into_bucket((stox['stox_1_roll'][stox.index[i]] - avg_1)/sd_1, a)
            zscores_1[a-1,i-1] = z_bucket
        else:
            zscores_1[a-1,i-1] = 'NaN'

        if stox['stox_2_roll'][stox.index[i]] != 'NaN':
            z_bucket = into_bucket((stox['stox_2_roll'][stox.index[i]] - avg_1)/sd_1, a)
            zscores_2[a-1,i-1] = z_bucket
        else:
            zscores_2[a-1,i-1] = 'NaN'

        if stox['stox_3_roll'][stox.index[i]] != 'NaN':
            z_bucket = into_bucket((stox['stox_3_roll'][stox.index[i]] - avg_1)/sd_1, a)
            zscores_3[a-1,i-1] = z_bucket
        else:
            zscores_3[a-1,i-1] = 'NaN'

In [54]:
zscores_1_names = [[str(a) for a in zscores_1[i,:]] for i in range(0,4)]
zscores_2_names = [[str(a) for a in zscores_2[i,:]] for i in range(0,4)]
zscores_3_names = [[str(a) for a in zscores_3[i,:]] for i in range(0,4)]

In [58]:
stox = pd.DataFrame({'Date': stox_1['Date'],
                     'zscore_1_1': zscores_1[0,:], 
                     'zscore_2_1': zscores_2[0,:],
                     'zscore_3_1': zscores_3[0,:],
                     'zscore_1_2': zscores_1[1,:], 
                     'zscore_2_2': zscores_2[1,:],
                     'zscore_3_2': zscores_3[1,:],
                     'zscore_1_3': zscores_1[2,:], 
                     'zscore_2_3': zscores_2[2,:],
                     'zscore_3_3': zscores_3[2,:],
                     'zscore_1_4': zscores_1[3,:], 
                     'zscore_2_4': zscores_2[3,:],
                     'zscore_3_4': zscores_3[3,:]})

In [59]:
stox.to_csv('stox_zscore_buckets.csv')  